# Model comparison 

This notebook is to compare all six models that have been investigated:
- K-neighbors 
- Decision Trees
- Logistic Regression 
- Linear SVM
- Random Forests 
- Multilayer perceptrons 

Definitions: using a confusion matrix the following measures of success can be calculated: 

TP = true positives
TN = true negatives 
FP = false positives 
FN = false negatives 

Accuracy: Overall how often is the classifier correct? (TP+TN)/total

Precision: When predicted yes, how often is it correct?
(TP/predicted yes)

Recall:  When it's actually yes, how often does it predict yes?
TP/actual yes 

f1_score: A weighted average of the true positive rate (recall) and precision 

ROC Curve: This is a commonly used graph that summarizes the performance of a classifier over all possible thresholds. It is generated by plotting the True Positive Rate (y-axis) against the False Positive Rate (x-axis) as you vary the threshold for assigning observations to a given class

In [ ]:
#import relevant libraries 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import preprocessing_ml as pp
import seaborn as sn
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from sklearn.metrics import classification_report
from sklearn.model_selection import cross_val_score

# import warnings filter
from warnings import simplefilter
# ignore all future warnings
simplefilter(action='ignore', category=FutureWarning)

import warnings
from sklearn.exceptions import DataConversionWarning,UndefinedMetricWarning
warnings.filterwarnings(action='ignore', category=DataConversionWarning)
warnings.filterwarnings(action='ignore', category=UndefinedMetricWarning)

In [ ]:
#import test dataset and framingham score system 
dataset = pd.read_csv('framingham.csv')
df_fram = pd.read_csv("Framingham Score/fram_risk.csv",index_col="index")[['TenYearCHD','fram_percent']]

In [ ]:
#performance algorithm from Logistic Regression kernal 
def cross_val(model, X, Y, scoring = ['accuracy','precision','recall','f1'], cv=5):
    
   #See https://scikit-learn.org/stable/modules/model_evaluation.html for list of allowed scoring params 
    from sklearn.model_selection import cross_val_score 
    
    print('\nCrossvalidation score for {} splits:\n'.format(cv))   
    
    cv_results = pd.DataFrame(columns=scoring)
    for measure in scoring:
        cv_results[measure] = cross_val_score(model, X, Y, scoring = measure, cv=cv)
    
    #print("Cross validation Accuracy:\n", cv_results.mean(),"\nCross validation STDEV:\n" , cv_results.std())
    return cv_results.mean()

def performance(model,X_test,y_test,cutoff=0.5,v=0,output='Accuracy'):
    from sklearn.preprocessing import binarize
    from sklearn.metrics import confusion_matrix, accuracy_score, f1_score, precision_score, recall_score, classification_report

    if model == 'fram': # cheap and dirty fix for framingham data needing different processing
        y_pred = X_test > cutoff*100 
    else:
        y_pred = model.predict_proba(X_test)
        y_pred = binarize(y_pred,cutoff)[:,1]
    
    if v == 1: 
        print('\n========\nCuttoff: ',cutoff)
        print('Confusion Matrix:')
        cm=confusion_matrix(y_test,y_pred)
        conf_matrix=pd.DataFrame(data=cm,columns=['Predicted:0','Predicted:1'],index=['Actual:0','Actual:1'])
        print(confusion_matrix(y_test,y_pred))
    
        print('Accuracy: {:.2f}%'.format(accuracy_score(y_test,y_pred)*100))
        print('f1 score: {:.2f}%'.format(f1_score(y_test,y_pred)*100))
        print('precision score: {:.2f}%'.format(precision_score(y_test,y_pred)*100))
        print('recall score: {:.2f}%'.format(recall_score(y_test,y_pred)*100))
        
        print(classification_report(y_test,y_pred))
    

    if output == 'Accuracy':
         return accuracy_score(y_test,y_pred)
    elif output == 'Precision':
         return precision_score(y_test,y_pred)
    elif output == 'Recall':
         return recall_score(y_test,y_pred)
    elif output == 'f1_score':
         return f1_score(y_test,y_pred)
    elif output == 'All':
        a = [accuracy_score(y_test,y_pred), precision_score(y_test,y_pred),
                      recall_score(y_test,y_pred), f1_score(y_test,y_pred)]
        return a




# Functions for each model:

In [ ]:
#function for the Framingham model currently used by doctors 
def fram_analysis(df_fram, v=0):
    range_x = np.arange(0.05, 1, 0.02)
    results_fram =[]
    
    for cutoff in range_x:
        results_fram.append(performance('fram', df_fram['fram_percent'], df_fram['TenYearCHD'], cutoff=cutoff, output='All'))
    results_fram = pd.DataFrame(results_fram, columns=['accuracy', 'precision', 'recall', 'f1'])
    
    return results_fram

In [ ]:
#function created with the optimal conditions for the k_neighbors algorithm - SMOTE still needed to add
def k_neighbors_opt(dataset, v=0):
    '''Do k_neighbors  fitting and print information about the success of the fitting
    dataset = dataset which will be used to train and test data
    v (optional, default = 0): int (0 or 1) verbose'''
    
    from sklearn.neighbors import KNeighborsClassifier
    print('\nOptimising parameters...\n')
    
    dataset_t = pp.chose_features(dataset, n_features=12) # choose n features
    dataset_t = pp.drop_missing(dataset_t) # drop missing rows
    dataset_t = pp.scale_data(dataset_t, 'standard', v=1 ) #scale data using minmax function 
    X_train, X_test, y_train, y_test = pp.smote(dataset_t) # split dataset
    
    print('\nCalculating K_neighbors ...\n')
    KN = KNeighborsClassifier(n_neighbors =16)
    KN.fit(X_train, y_train)
    y_pred = KN.predict(X_test)
    
    if v == 1:
        print('\nRunning K_neighbors algorithm...\n')
        accuracy = accuracy_score(y_test, y_pred)
        print('\nAccuracy is:', accuracy)
        classification = classification_report(y_test, y_pred)
        print(classification)
        
        print('Calculating metrics for ranging cutoff...')
        range_x = np.arange(0.05, 1, 0.02)
        results_KN = []

        for cutoff in range_x:
            results_KN.append(performance(KN, X_test, y_test,cutoff=cutoff, output='All'))
        results_KN = pd.DataFrame(results_KN,columns=['accuracy','precision','recall','f1'])
    
    #probability score for ROC curve:
    y_pred_prob_yes=KN.predict_proba(X_test)
    
    #confusion matrix:
    cm=confusion_matrix(y_test,y_pred)


    return results_KN, y_pred_prob_yes, y_test, cm
    

In [ ]:
#function created with optimised SVM algorithm - SMOTE still needs to be added 
def SVM_opt(dataset, v=0):
    '''Do linear support vector machine fitting and print information about the success of the fitting 
    dataset = dataset which will be used to train and test data 
    v (optional, default =0): int (0 or 1) verbose'''
    
    print('\nOptimising parameters...\n')
    dataset_t = pp.chose_features(dataset, n_features=12)
    dataset_t = pp.drop_missing(dataset_t) # drop missing rows
    dataset_t = pp.scale_data(dataset_t, 'minmax', v=1)
    X_train, X_test, y_train, y_test = pp.smote(dataset_t) # split dataset
    
    from sklearn.svm import SVC
    
    print('\nCalculating Support vector machine ..\n')
    SVM = SVC(C=3, random_state=0, probability=True)
    SVM.fit(X_train, y_train) 
    y_pred = SVM.predict(X_test)
    
    if v == 1: 

        print('Running support vector machine...')
        accuracy = accuracy_score(y_test, y_pred)
        print('\nAccuracy is:', accuracy)
        classification = classification_report(y_test, y_pred)
        print(classification)
        
        print('\nCalculating metrics for ranging classification...')
        range_x = np.arange(0.05, 1, 0.02)
        results_SVM =[]
        for cutoff in range_x:
            results_SVM.append(performance(SVM, X_test, y_test,cutoff=cutoff, output='All'))
        results_SVM = pd.DataFrame(results_SVM,columns=['accuracy','precision','recall','f1']) 

    #probability score for ROC curve:    
    y_pred_prob_yes=SVM.predict_proba(X_test)
    
    #Confusion matrix:
    cm=confusion_matrix(y_test,y_pred)


    return results_SVM, y_pred_prob_yes, y_test, cm

In [ ]:
#function created with optimised MLP algorithm - SMOTE still needed to add
def multilayer_opt(dataset, v=0):
    '''Do multilayer perceptron fitting and print information about the success of the fitting
    dataset = dataset which will be used to train and test data
    v (optional, default = 0): int (0 or 1) verbose'''
    print('\nOptimising parameters...\n')
    dataset_t = pp.chose_features(dataset, n_features=12) # choose n features
    dataset_t = pp.drop_missing(dataset_t) # drop missing rows
    dataset_t = pp.scale_data(dataset_t, 'standard', v=1)
    X_train, X_test, y_train, y_test = pp.smote(dataset_t) # split dataset
    
    print('\nCalculating multilayer perceptron ...\n')
    
    from sklearn.neural_network import MLPClassifier
    MLP = MLPClassifier(hidden_layer_sizes=(1,), activation= 'identity', solver='sgd', alpha=0.05, random_state=0)
    MLP.fit(X_train, y_train) 
    y_pred = MLP.predict(X_test)
    
    if v == 1:
        print('Running multilayer perceptrons...')
        accuracy = accuracy_score(y_test, y_pred)
        print('\nAccuracy is:', accuracy)
        classification = classification_report(y_test, y_pred)
        print(classification)
        
        print('Calculating metrics for ranging cutoff...')
        results_MLP = []
        range_x = np.arange(0.05, 1, 0.02)
        for cutoff in range_x:
            results_MLP.append(performance(MLP, X_test, y_test,cutoff=cutoff, output='All'))
        results_MLP = pd.DataFrame(results_MLP,columns=['accuracy','precision','recall','f1'])
        

    #probability score for ROC curve:    
    y_pred_prob_yes=MLP.predict_proba(X_test)
    
    #confusion matrix:
    cm=confusion_matrix(y_test,y_pred)


    return results_MLP, y_pred_prob_yes, y_test, cm
    

In [ ]:
#function created with optimised RF algorithm 
def RF_opt(dataset, v = 0, r_state = 0):
    '''Random forest classifier used to machine learn, fit the data and print information about the success of the algorithm.
    dataset = dataset which will be used to train and test data
    v (optional, default = 0): int (0 or 1) verbose'''
    print('\nOptimising parameters...\n')
    features = ['sysBP', 'age', 'cigsPerDay', 'totChol', 'diaBP', 'prevalentHyp', 'diabetes', 'BPMeds', 'male', 'BMI', 'TenYearCHD']
    #dataset_t = pp.outliers(dataset)
    dataset_t = pp.chose_features(dataset, features = features, n_features = -1)
    dataset_t = pp.drop_missing(dataset_t)
    X_train, X_test, y_train, y_test = pp.smote(dataset_t) # split dataset
    
    print('\nCalculating Random Forests ...\n')
    
    from sklearn.ensemble import RandomForestClassifier #Importing random forest module
    from sklearn.preprocessing import binarize
    
    RF = RandomForestClassifier(n_estimators = 120, bootstrap = True, random_state = r_state)
    RF.fit(X_train, y_train)
    
    cut_off = 0.35
    
    RF_preds = RF.predict_proba(X_test)[:, 1]
    y_pred = (RF_preds >= cut_off).astype('int')
    
    if v == 1:
        print('Running Random Forests ...')
        accuracy = accuracy_score(y_test, y_pred)
        print('\nAccuracy is:', accuracy)
        classification = classification_report(y_test, y_pred)
        print(classification)
        
        print('Calculating metrics for ranging cutoff...')
        results_RF = []
        range_x = np.arange(0.05, 1, 0.02)
        for cutoff in range_x:
            results_RF.append(performance(RF, X_test, y_test,cutoff=cutoff, output='All'))
        results_RF = pd.DataFrame(results_RF,columns=['accuracy','precision','recall','f1'])
        

    #probability score for ROC curve:    
    y_pred_prob_yes=RF.predict_proba(X_test)
    
    #confusion matrix:
    cm=confusion_matrix(y_test,y_pred)


    return results_RF, y_pred_prob_yes, y_test, cm

In [ ]:
#function created with optimised RF algorithm 
def DT_opt(dataset, v = 0, r_state = 0):
    '''Decision Tree classifier used to machine learn, fit the data and print information about the success of the algorithm.
    dataset = dataset which will be used to train and test data
    v (optional, default = 0): int (0 or 1) verbose'''
    print('\nOptimising parameters...\n')
    features = ['sysBP', 'age', 'cigsPerDay', 'totChol', 'diaBP', 'prevalentHyp', 'diabetes', 'BPMeds', 'male', 'BMI', 'TenYearCHD']
    #dataset_t = pp.outliers(dataset)
    dataset_t = pp.chose_features(dataset, features = features, n_features = -1)
    dataset_t = pp.drop_missing(dataset_t)
    X_train, X_test, y_train, y_test = pp.smote(dataset_t) # split dataset
    
    print('\nCalculating Decision Trees ...\n')
    
    from sklearn.tree import DecisionTreeClassifier #Importing decision tree classifier
    from sklearn.preprocessing import binarize
    
    DT = DecisionTreeClassifier(random_state = r_state)
    DT.fit(X_train, y_train)
    
    cut_off = 0.35
    
    DT_preds = DT.predict_proba(X_test)[:, 1]
    y_pred = (DT_preds >= cut_off).astype('int')
    
    if v == 1:
        print('Running Decision Trees ...')
        accuracy = accuracy_score(y_test, y_pred)
        print('\nAccuracy is:', accuracy)
        classification = classification_report(y_test, y_pred)
        print(classification)
        
        print('Calculating metrics for ranging cutoff...')
        results_DT = []
        range_x = np.arange(0.05, 1, 0.02)
        for cutoff in range_x:
            results_DT.append(performance(DT, X_test, y_test, cutoff=cutoff, output='All'))
        results_DT = pd.DataFrame(results_DT,columns=['accuracy','precision','recall','f1'])
        

    #probability score for ROC curve:    
    y_pred_prob_yes=DT.predict_proba(X_test)
    
    #confusion matrix:
    cm=confusion_matrix(y_test,y_pred)


    return results_DT, y_pred_prob_yes, y_test, cm

In [ ]:
#function created with optimised RF algorithm 
def LR_opt(dataset, v = 0, r_state = 0):
    '''Logistic Regression used to machine learn, fit the data and print information about the success of the algorithm.
    dataset = dataset which will be used to train and test data
    v (optional, default = 0): int (0 or 1) verbose'''
    print('\nOptimising parameters...\n')
    
    from statsmodels.tools import add_constant as add_constant
    dataset_t = add_constant(dataset)

    features = ['TenYearCHD','const','sysBP', 'glucose', 'age', 'totChol', 'cigsPerDay', 'diaBP', 'prevalentHyp',
                'diabetes', 'BPMeds', 'male', 'BMI',
                'education', 'heartRate'] 
    #dataset_t = pp.outliers(dataset)
    dataset_t = pp.chose_features(dataset_t, features = features)
    dataset_t = pp.drop_missing(dataset_t)
    dataset_t = pp.scale_data(dataset_t, 'standard')
    X_train, X_test, y_train, y_test = pp.split_data(dataset_t,r_state=r_state) # split dataset
    
    print('\nCalculating Logistic Regression ...\n')
    
    from sklearn.linear_model import LogisticRegression
    from sklearn.preprocessing import binarize
    
    LR = LogisticRegression(random_state=r_state).fit(X_train, y_train) 
    
    cutoff = 0.1840
    y_pred = LR.predict_proba(X_test)
    y_pred = binarize(y_pred,cutoff)[:,1]
    
    if v == 1:
        print('Running Logistic Regression ...')
        accuracy = accuracy_score(y_test, y_pred)
        print('\nAccuracy is:', accuracy)
        classification = classification_report(y_test, y_pred)
        print(classification)
        
        print('Calculating metrics for ranging cutoff...')
        results_LR = []
        range_x = np.arange(0.05, 1, 0.02)
        for cutoff in range_x:
            results_LR.append(performance(LR, X_test, y_test, cutoff=cutoff, output='All'))
        results_LR = pd.DataFrame(results_LR,columns=['accuracy','precision','recall','f1'])
        

    #probability score for ROC curve:    
    y_pred_prob_yes=LR.predict_proba(X_test)
    
    #confusion matrix:
    cm=confusion_matrix(y_test,y_pred)


    return results_LR, y_pred_prob_yes, y_test, cm

In [ ]:
#Run all alogrithms on dataset 
resultsLR, ROC_LR, y_test_LR, cm_LR = LR_opt(dataset, v=1)
resultsKN, ROC_KN, y_test_KN, cm_KN = k_neighbors_opt(dataset, v=1)
resultsSVM, ROC_SVM, y_test_SVM, cm_SVM = SVM_opt(dataset, v=1)
resultsMLP, ROC_MLP, y_test_MLP, cm_MLP = multilayer_opt(dataset, v=1)
resultsRF, ROC_RF, y_test_RF, cm_RF = RF_opt(dataset, v=1)
resultsDT, ROC_DT, y_test_DT, cm_DT = DT_opt(dataset, v=1)



results_fram = fram_analysis(df_fram)




# Comparisons: accuracy, precision, recall, f1_score, roc curve

In [ ]:
#PLOTS 
plt.figure(figsize=(20, 10))
range_x = np.arange(0.05, 1, 0.02)
plt.subplot(2,2,1)
plt.plot(range_x, resultsMLP['accuracy'], label='MLP')
plt.plot(range_x, resultsSVM['accuracy'], label='SVM')
plt.plot(range_x, resultsKN['accuracy'], label ='KN')
plt.plot(range_x, resultsRF['accuracy'], label ='RF')
plt.plot(range_x, resultsDT['accuracy'], label ='DT')
plt.plot(range_x, resultsLR['accuracy'], label ='LR')
plt.plot(range_x, results_fram['accuracy'], label ='fram' )


plt.grid()
plt.title('Changing cutoff to see effect on accuracy')
plt.xlabel('Cutoff')
plt.ylabel('Accuracy Score')


plt.subplot(2,2,2)
#Precision Graph 
plt.plot(range_x, resultsMLP['precision'], label='MLP')
plt.plot(range_x, resultsSVM['precision'], label='SVM')
plt.plot(range_x, resultsKN['precision'], label ='KN') 
plt.plot(range_x, resultsRF['precision'], label ='RF')
plt.plot(range_x, resultsDT['precision'], label ='DT')
plt.plot(range_x, resultsLR['precision'], label ='LR')
plt.plot(range_x, results_fram['precision'], label ='fram' )

plt.grid()
plt.title('Changing cutoff to see effect on Precision')
plt.xlabel('Cutoff')
plt.ylabel('Precision Score')
plt.legend(loc='center left', bbox_to_anchor=(1, 0.9))

#Recall Graph 
plt.subplot(2,2,3)
plt.plot(range_x, resultsMLP['recall'], label='MLP')
plt.plot(range_x, resultsSVM['recall'], label='SVM')
plt.plot(range_x, resultsKN['recall'], label ='KN')
plt.plot(range_x, resultsRF['recall'], label ='RF')
plt.plot(range_x, resultsDT['recall'], label ='DT')
plt.plot(range_x, resultsLR['recall'], label ='LR')
plt.plot(range_x, results_fram['recall'], label ='fram' )

plt.grid()
plt.title('Changing cutoff to see effect on Recall')
plt.xlabel('Cutoff')
plt.ylabel('Recall Score')

#F1_score Graph 
plt.subplot(2,2,4)
plt.plot(range_x, resultsMLP['f1'], label='MLP')
plt.plot(range_x, resultsSVM['f1'], label='SVM')
plt.plot(range_x, resultsKN['f1'], label ='KN')
plt.plot(range_x, resultsRF['f1'], label ='RF')
plt.plot(range_x, resultsDT['f1'], label ='DT')
plt.plot(range_x, resultsLR['f1'], label ='LR')
plt.plot(range_x, results_fram['f1'], label ='fram' )

plt.grid()
plt.title('Changing cutoff to see effect on F1_score')
plt.xlabel('Cutoff')
plt.ylabel('F1 Score')


# ROC curve

In [ ]:
#ROC CURVE
plt.figure(figsize = (8,5))


#K-Neighbors
fpr, tpr, thresholds = roc_curve(y_test_KN, ROC_KN[:,1])
plt.plot(fpr,tpr,label ='KN')
print('roc auc for KN:',roc_auc_score(y_test_KN,ROC_KN[:,1]))

#SVM
fpr, tpr, thresholds = roc_curve(y_test_SVM, ROC_SVM[:,1])
plt.plot(fpr,tpr,label ='SVM')
print('roc auc for SVM:',roc_auc_score(y_test_SVM,ROC_SVM[:,1]))


#MLP
fpr, tpr, thresholds = roc_curve(y_test_MLP, ROC_MLP[:,1])
plt.plot(fpr,tpr,label ='MLP')
print('roc auc for MLP:',roc_auc_score(y_test_MLP,ROC_MLP[:,1]))

#RF
fpr, tpr, thresholds = roc_curve(y_test_RF, ROC_RF[:,1])
plt.plot(fpr,tpr,label ='RF')
print('roc auc for RF:',roc_auc_score(y_test_RF,ROC_RF[:,1]))

#DT
fpr, tpr, thresholds = roc_curve(y_test_DT, ROC_DT[:,1])
plt.plot(fpr,tpr,label ='DT')
print('roc auc for DT:',roc_auc_score(y_test_DT,ROC_DT[:,1]))

#DT
fpr, tpr, thresholds = roc_curve(y_test_LR, ROC_LR[:,1])
plt.plot(fpr,tpr,label ='LR')
print('roc auc for LR:',roc_auc_score(y_test_LR,ROC_LR[:,1]))

#framingham dataset 
fpr, tpr, thresholds = roc_curve(df_fram.TenYearCHD, df_fram.fram_percent)
plt.plot(fpr,tpr,label ='fram')
print('roc auc for framingham:',roc_auc_score(df_fram.TenYearCHD, df_fram.fram_percent))



plt.plot([0, 1], [0, 1], linestyle='--', lw=2, color='r',
         label='Chance', alpha=.8)
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.0])
plt.legend()
plt.grid()
plt.title('ROC curves for all optimised algorithms')
plt.xlabel('False positive rate (1-Specificity)')
plt.ylabel('True positive rate (Sensitivity)')


# Confusion matrix for all alogrithms

In [ ]:
cm_KN  = pd.DataFrame(data=cm_KN,columns=['Predicted:0','Predicted:1'],index=['Actual:0','Actual:1'])
cm_SVM = pd.DataFrame(data=cm_SVM,columns=['Predicted:0','Predicted:1'],index=['Actual:0','Actual:1'])
cm_MLP = pd.DataFrame(data=cm_MLP,columns=['Predicted:0','Predicted:1'],index=['Actual:0','Actual:1'])
cm_RF = pd.DataFrame(data=cm_RF,columns=['Predicted:0','Predicted:1'],index=['Actual:0','Actual:1'])
cm_LR = pd.DataFrame(data=cm_LR,columns=['Predicted:0','Predicted:1'],index=['Actual:0','Actual:1'])


plt.figure(figsize=(20,20))
plt.subplot(3,2,1)
sn.heatmap(cm_KN, annot=True, fmt='d', cmap="YlGnBu")
plt.title('KN ')

plt.subplot(3,2,2)
sn.heatmap(cm_SVM, annot=True,fmt='d',cmap="YlGnBu")
plt.title('SVM ')


plt.subplot(3,2,3)
sn.heatmap(cm_MLP, annot=True,fmt='d',cmap="YlGnBu")
plt.title('MLP ')

plt.subplot(3,2,4)
sn.heatmap(cm_RF, annot=True,fmt='d',cmap="YlGnBu")
plt.title('RF ')

plt.subplot(3,2,5)
sn.heatmap(cm_LR, annot=True,fmt='d',cmap="YlGnBu")
plt.title('LR ')



In [ ]:
results_fram = fram_analysis(df_fram)